In [2]:
!pip3 install tensorflow-gpu==2.0.0.rc1
!pip3 install tqdm

  ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.0.0.rc1 (from versions: none)
ERROR: No matching distribution found for tensorflow-gpu==2.0.0.rc1
     |████████████████████████████████| 61kB 1.4MB/s eta 0:00:01
